# Identifying the Four Morphological Types

## 1. Module Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from matplotlib.ticker import MaxNLocator
from magnetar.funcs import init_conds, ODEs

## 2. Defining Global Variables

In [ ]:
# Global constants
G = 6.674e-8                      # Gravitational constant - cgs units
c = 3.0e10                        # Speed on light - cm/s
R = 1.0e6                         # Radius of Magnetar - km
Msol = 1.99e33                    # Solar mass - grams
M = 1.4 * Msol                    # Mass of Magnetar - grams
I = (4.0 / 5.0) * M * (R ** 2.0)  # Moment of Inertia
GM = G * M
alpha = 0.1  # Sound speed prescription
cs7 = 1.0    # Speed of sound - cm/s
k = 0.9      # Capping fraction

t = np.logspace(0.0, 6.0, num=10001, base=10.0)  # Time array

# Dictionary containing parameters describing 4 morphological types
# Parameter order: B, P, MdiscI, RdiscI, epsilon, delta, 
pars = {    "humped" : [1.0, 10.0,  1.0e-3, 100.0,  1.0,  1.0, 1.0],
           "classic" : [1.0,  5.0,  1.0e-3, 500.0,  1.0,  1.0, 1.0],
            "sloped" : [1.0,  1.0,  1.0e-2, 100.0, 10.0, 10.0, 1.0],
        "stuttering" : [1.0, 10.0,  1.0e-4, 100.0,  1.0, 10.0, 1.0]}

In [ ]:
# Create figure with 7 subplots all sharing x-axis
fig, axes = plt.subplots(4, 3, sharex=True, figsize=(7,6))

for i, key in enumerate(pars.keys()):
    # Get parameters from dictionary
    B, P, MdiscI, RdiscI, epsilon, delta, n = pars[key]

    # Convert units to cgs
    Rdisc = RdiscI * 1.0e5                 # Convert radius to cm
    tvisc = Rdisc / (alpha * cs7 * 1.0e7)  # Viscous timescale
    mu = 1.0e15 * B * (R ** 3.0)           # Magnetic Dipole Moment
    M0 = delta * MdiscI * Msol             # Global Mass Budget
    tfb = epsilon * tvisc                  # Fallback timescale

    y0 = init_conds(P, MdiscI)  # Initial conditions
    
    # Solve the ODEs
    soln = odeint(ODEs, y0, t, args=(B, MdiscI, RdiscI, epsilon, delta, n))
    Mdisc = soln[:,0]
    omega = soln[:,1]
    
    # Calculate Radii
    Rm = ((mu ** (4.0 / 7.0)) * (GM ** (-1.0 / 7.0)) * ((Mdisc / tvisc) **
          (-2.0 / 7.0)))
    Rc = (GM / (omega ** 2.0)) ** (1.0 / 3.0)
    Rlc = c / omega
    Rm = np.where(Rm >= (k * Rlc), (k * Rlc), Rm)
    
    # Plot the Radii
    axes[i,2].axhline(RdiscI, c='k', ls='-.')      # Disc
    axes[i,2].axhline(R/1.0e5, c='k', ls='-.')     # Magnetar
    axes[i,2].loglog(t, Rc/1.0e5, c='k', ls=':')   # Corotation
    axes[i,2].loglog(t, Rm/1.0e5, c='k', ls='--')  # Alfven
    axes[i,2].loglog(t, Rlc/1.0e5, c='k')          # Light cylinder
    axes[i,2].set_xlim(1.0e0, 1.0e6)
    axes[i,2].set_ylim(1.0e0, 1.0e4)
    axes[i,2].tick_params(axis='both', which='major', labelsize=10)

    w = (Rm / Rc) ** (3.0 / 2.0)      # Fastness parameter
    bigT  = 0.5 * I * (omega ** 2.0)  # Rotational energy
    modW = (0.6 * M * (c ** 2.0) * ((GM / (R * (c ** 2.0))) / (1.0 - 0.5 * (GM
            / (R * (c ** 2.0))))))    # Binding energy
    rot_param = bigT / modW           # Rotation parameter
    
    # Dipole torque
    Ndip = (-1.0 * (mu ** 2.0) * (omega ** 3.0)) / (6.0 * (c ** 3.0))
    
    # Efficiencies and mass flow rates
    eta2 = 0.5 * (1.0 + np.tanh(n * (w - 1.0)))
    eta1 = 1.0 - eta2
    Mdotprop = eta2 * (Mdisc / tvisc)  # Propelled
    Mdotacc = eta1 * (Mdisc / tvisc)   # Accreted

    # Plot the Mass flows
    axes[i,1].semilogx(t, 1.0e4*(Mdotacc/Msol), c='k')
    axes[i,1].semilogx(t, 1.0e4*(Mdotprop/Msol), c='k', ls='--')
    axes[i,1].set_xlim(1.0e0, 1.0e6)
    axes[i,1].tick_params(axis='both', which='major', labelsize=10)
    
    # Accretion torque
    Nacc = np.zeros_like(Ndip)
    Nacc = np.where(Rm >= R, ((GM * Rm) ** 0.5) * (Mdotacc - Mdotprop), Nacc)
    Nacc = np.where(Rm < R, ((GM * R) ** 0.5) * (Mdotacc - Mdotprop), Nacc)
    Nacc = np.where(rot_param >= 0.27, 0.0, Nacc)

    # Dipole Luminosity
    Ldip = ((mu ** 2.0) * (omega ** 4.0)) / (6.0 * (c ** 3.0))
    inLdip2 = np.isfinite(Ldip)
    Ldip = np.where(np.isfinite(Ldip), Ldip, 0.0)
    
    # Propeller luminosity
    Lprop = (-1.0 * Nacc * omega) - ((GM / Rm) * eta2 * (Mdisc / tvisc))
    Lprop = np.where(Lprop <= 0.0, 0.0, Lprop)
    Lprop = np.where(np.isfinite(Lprop), Lprop, 0.0)
    
    # Total luminosity
    Ltot = Ldip + Lprop
    
    # Plot the luminosities
    axes[i,0].loglog(t, Ldip/1.0e50, c='k', ls=':')
    axes[i,0].loglog(t, Lprop/1.0e50, c='k', ls='--')
    axes[i,0].loglog(t, Ltot/1.0e50, c='k')
    axes[i,0].set_xlim(1.0e0, 1.0e6)
    axes[i,0].set_ylim(1.0e-7, 1.0e0)
    axes[i,0].tick_params(axis='both', which='major', labelsize=10)

# Plot formatting
# Set y-axis tick marks
for i in range(4):
    axes[i,0].set_yticks([1.0e-6, 1.0e-4, 1.0e-2, 1.0e0])
    axes[i,1].yaxis.set_major_locator(MaxNLocator(5, prune='lower'))
    axes[i,2].set_yticks([1.0e1, 1.0e2, 1.0e3, 1.0e4])

axes[2,0].set_ylim(1.0e-7, 1.0e2)
axes[2,0].set_yticks([1.0e-6, 1.0e-4, 1.0e-2, 1.0e0, 1.0e2])

# Set x-axis tick marks
for i in range(3):
    axes[-1,i].set_xticks([1.0e0, 1.0e2, 1.0e4, 1.0e6])

# Add text
fig.tight_layout(h_pad=0.0)
fig.text(0.5, 0.00, 'Time (s)', ha='center', fontsize=12)
fig.text(0.0, 0.5, 'Luminosity ($10^{50}$ erg ${\\rm s}^{-1}$)',
         fontsize=12, va='center', rotation='vertical')
fig.text(0.33, 0.5, '$10^{-4}$ $M_{\odot}$ ${\\rm s}^{-1}$', fontsize=12,
         va='center', rotation='vertical')
fig.text(0.66, 0.5, 'Radial distance (km)', fontsize=12, va='center',
         rotation='vertical');

<div class="alert alert-block alert-warning">
<b>Warning:</b> This plot is currently not reproducible from the paper.
</div>